In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torchvision import datasets
from torchvision import transforms
from torchmetrics.classification import MulticlassAccuracy
from torchinfo import summary

In [2]:
no_epochs = 10
learning_rate = 0.001
batch_size = 128

acc_function = MulticlassAccuracy(num_classes=102, average='micro')
loss_fn = nn.CrossEntropyLoss()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Data Augmentation
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Assuming you want to keep the default transformations for testing/validation:
default_transforms = transforms.Compose([
    models.VGG16_BN_Weights.IMAGENET1K_V1.transforms()
])

flowers_train = datasets.Flowers102(root='./data', split='train', download=True, transform=train_transforms)
flowers_test = datasets.Flowers102(root='./data', split='test', download=True, transform=default_transforms)
flowers_val = datasets.Flowers102(root='./data', split='val', download=True, transform=default_transforms)

# transformation = transforms.Compose([
#     models.VGG16_BN_Weights.IMAGENET1K_V1.transforms()
# ])

# flowers_train = datasets.Flowers102(root='./data', split='train', download=True, transform=transformation)
# flowers_test = datasets.Flowers102(root='./data', split='test', download=True, transform=transformation)
# flowers_val = datasets.Flowers102(root='./data', split='val', download=True, transform=transformation)

In [4]:
def get_data_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(flowers_train, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(flowers_test, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(flowers_val, batch_size=batch_size, shuffle=True)
    return train_loader, test_loader, val_loader

In [5]:
def train(model, optimizer, dataloader, loss_fn=loss_fn):
    running_loss_value = 0
    for images, labels in dataloader:
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        running_loss_value += loss.item()
        print(loss.item())
        loss.backward()
        optimizer.step()
    return running_loss_value / len(dataloader)

def test_eval(model, dataloader, loss_fn=loss_fn):
    running_loss_value = 0
    running_acc_value = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            acc = acc_function(outputs, labels)
            running_loss_value += loss.item()
            running_acc_value += acc.item()
            print(loss.item(), acc.item())
    running_acc_value /= len(dataloader)
    running_loss_value /= len(dataloader)
    return running_acc_value, running_loss_value

def train_eval_loop(model, train_dataloader, val_dataloader, no_epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr = [], [], [], []
    for i in range(no_epochs):
        print(f'Epoch {i+1}')
        train_loss = train(model, optimizer, train_dataloader)
        eval_loss, eval_acc = test_eval(model, val_dataloader)
        print('Train Loss: {}, Eval Accuracy: {}, Eval Loss: {}'.format(train_loss, eval_acc, eval_loss))
        train_loss_arr.append(train_loss)
        eval_loss_arr.append(eval_loss)
        eval_acc_arr.append(eval_acc)
    return train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr

In [6]:
def create_model():
    model = models.vgg16_bn(weights=models.VGG16_BN_Weights.DEFAULT)
    new_classifier_head = nn.Sequential(
        nn.Linear(25088, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 102)
    )
    for param in model.features.parameters():
        param.requires_grad = False
    model.classifier = new_classifier_head
    
    # for l in model.classifier:
    #     for p in l.parameters():
    #         p.requires_grad = True
    return model.to(device=device)

In [7]:
model = create_model()
acc_function = acc_function.to(next(model.parameters()).device)
train_data_loader, test_data_loader, val_data_loader = get_data_loader(batch_size)
train_acc, train_loss, eval_acc, eval_loss = train_eval_loop(model, train_data_loader, val_data_loader, no_epochs=no_epochs)

Epoch 1
4.639930725097656
5.5722575187683105
5.447599411010742
5.597398281097412
5.780558109283447
4.923162937164307
4.95997953414917
4.645816802978516
4.4334516525268555 0.1171875
4.288261890411377 0.1484375
4.353222370147705 0.1484375
4.296097278594971 0.09375
4.802391529083252 0.1015625
4.186879634857178 0.15625
4.195619106292725 0.0859375
3.926811933517456 0.14516128599643707
Train Loss: 5.195837914943695, Eval Accuracy: 4.31034192442894, Eval Loss: 0.12459047324955463
Epoch 2
4.287167072296143
3.9972941875457764
4.181175231933594
3.5606110095977783
3.1730711460113525
3.1930041313171387
2.991826295852661
2.999133348464966
2.5620901584625244 0.4296875
2.6098883152008057 0.4453125
2.6115405559539795 0.390625
2.745239734649658 0.40625
2.386171817779541 0.4921875
2.636338710784912 0.3828125
2.4379613399505615 0.4453125
2.5656561851501465 0.44354838132858276
Train Loss: 3.547910302877426, Eval Accuracy: 2.569360852241516, Eval Loss: 0.42946698516607285
Epoch 3
2.7004833221435547
2.61553